# Example of shear-wave splitting applied to an icequake from Rutford Ice Stream, Antarctica

In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import swspy
import obspy
from obspy import UTCDateTime
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

In [182]:
# Load data:
archive_path = "data/mseed"
archive_vs_file = "archive"
nonlinloc_event_path = "data/loc.Tom__RunNLLoc000.20090121.042009.grid0.loc.hyp"

starttime = UTCDateTime("20090121T042009.18523") - 0.5
endtime = UTCDateTime("20090121T042009.18523") + 2.5
load_wfs_obj = swspy.io.load_waveforms(archive_path, starttime, endtime)
load_wfs_obj.filter = True
load_wfs_obj.filter_freq_min_max = [1.0, 50.0]
st = load_wfs_obj.read_waveform_data()

In [183]:
# Add noise to data:
for i in range(len(st)):
    st[i].data = st[i].data + 0.3*np.roll(st[i].data, int(np.random.uniform(-100, 100,1))) + 0.3*np.roll(st[i].data, int(np.random.uniform(-100, 100,1)))

In [186]:
# Calculate splitting:
splitting_event = swspy.splitting.create_splitting_object(st.select(station="ST01"), nonlinloc_event_path)
splitting_event.overall_win_start_pre_fast_S_pick = 0.2 #0.1
splitting_event.overall_win_start_post_fast_S_pick = 0.4 #0.2
splitting_event.rotate_step_deg = 1.0 #2.5
splitting_event.max_t_shift_s = 0.12
splitting_event.n_win = 5 #5 #10
# splitting_event.perform_sp
splitting_event.perform_sws_analysis()

NameError: name 'labels' is not defined

In [120]:
X = np.array([[1, 2], [2, 2], [2, 3],[8, 7], [8, 8], [25, 80]])

In [121]:
X.shape

(6, 2)

In [126]:
a = np.arange(6)
b = np.arange(6)
Y = np.dstack((a,b))[0,:,:]
Y.shape

(6, 2)

array([ 56.7081792])